# LGSynth'91 benchmarks comparison
tested benchmarks: C432, C499, C880, C1355, C1908

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# import matplotlib.ticker as tick

rc('font',**{'family':'serif','serif':['Helvetica']})
rc('text', usetex=True)

In [68]:
blif_nodecount_csv = "./csv/blif-nc.csv"
blif_boxcount_csv = "./csv/blif-boxes.csv"

blif_bc_output = "./figures/blif-box-counts.pdf"
blif_nc_output = "./figures/blif-comparison.pdf"

In [69]:
df = pd.read_csv(blif_nodecount_csv)
print(df['benchmark'].size)
df = df[~df['benchmark'].str.startswith('C499')]
print(df['benchmark'].size)
df = df.reset_index(drop=True)
df.rename(columns={'esr': 'esrbdd'}, inplace=True)

103
71


In [70]:
def getStatsDataframe(df: pd.DataFrame):
    data = df.drop(columns=['benchmark'])
    result = {'model': [], 'sum': [], 'mean': []}

    for col in data.columns:
        column_sum = df[col].sum()
        column_mean = df[col].mean()
        result['model'].append(col)
        result['sum'].append(column_sum)
        result['mean'].append(column_mean)

    return pd.DataFrame(result)

In [71]:
def getStatsByBenchmark(df: pd.DataFrame, order):
    data = df.copy()
    data = data.reindex(columns=order)
    data['benchmark'] = data['benchmark'].str.split('.').str[0]
    aggregate_df = data.groupby('benchmark').sum()
    aggregate_df = aggregate_df.drop(columns=['init', 'unfold', 'normal'])
    result = aggregate_df.reset_index()
    return result

In [72]:
def getMaxBenchmarkStats(df: pd.DataFrame, order):
    data = df.copy()
    data = data.reindex(columns=order)
    data['benchmark'] = data['benchmark'].str.split('.').str[0]
    dfs = dict(tuple(data.groupby('benchmark')))

    result = {'benchmark': data.columns.tolist()[1:]}
    for benchmark, dataframe in dfs.items():
        result[benchmark] = dataframe.loc[dataframe['init'].idxmax()].values.tolist()[1:]
    result = pd.DataFrame(result)
    result = pd.DataFrame(result).transpose().reset_index()

    new_header = result.iloc[0]
    result = result[1:]
    result = result.set_axis(new_header, axis=1, copy=False).reset_index(drop=True)
    result = result.drop(columns=['init', 'unfold', 'normal'])
    result = result.set_index(['benchmark'])
    return result

In [73]:
def computeAvgReduction(data: pd.DataFrame, reference: str, target: str):
    reductions = []
    for i in range(len(data[reference])):
        ref = data[reference][i]
        val = data[target][i]
        res = (ref - val) / ref
        reductions.append(res)
    return round((sum(reductions) / len(reductions)) * 100, ndigits=3)

In [74]:
print('(bdd-abdd) / bdd = (%)', computeAvgReduction(df, 'bdd', 'abdd'))
print('(zbdd-abdd) / zbdd = (%)', computeAvgReduction(df, 'zbdd', 'abdd'))
print('(tbdd-abdd) / tbdd = (%)', computeAvgReduction(df, 'tbdd', 'abdd'))
print('(esrbdd-abdd) / esrbdd = (%)', computeAvgReduction(df, 'esrbdd', 'abdd'))
print('(czdd-abdd) / czdd = (%)', computeAvgReduction(df, 'czdd', 'abdd'))
print('(cbdd-abdd) / cbdd = (%)', computeAvgReduction(df, 'cbdd', 'abdd'))

(bdd-abdd) / bdd = (%) 16.881
(zbdd-abdd) / zbdd = (%) 20.891
(tbdd-abdd) / tbdd = (%) 16.857
(esrbdd-abdd) / esrbdd = (%) 14.44
(czdd-abdd) / czdd = (%) 13.92
(cbdd-abdd) / cbdd = (%) 8.589


In [75]:
order = ['benchmark', 'init', 'unfold', 'normal', 'bdd', 'zbdd', 'tbdd', 'czdd', 'cbdd', 'esrbdd', 'abdd']
benchmarks_sum = getStatsByBenchmark(df, order)
benchmarks_sum.columns = benchmarks_sum.columns.str.upper()
benchmarks_sum.loc['Total'] = benchmarks_sum.sum()
# benchmarks_sum.to_latex("benchmark-sum.nodecounts.tex")

In [76]:
box_df = pd.read_csv(blif_boxcount_csv)
box_df = box_df[~box_df['benchmark'].str.startswith('C499')]
box_df = box_df.reset_index(drop=True)
box_df = box_df.drop(columns=['benchmark', 'norm', 'full'])
box_df.loc['Total'] = box_df.sum()
all_box_usage_total = box_df.iloc[-1].sum()

box_df.columns = ['$\mathtt{X}$', '$\mathtt{L}_\mathbf{0}$', '$\mathtt{L}_\mathbf{1}$', '$\mathtt{L}_\oplus$', '$\mathtt{H}_\mathbf{0}$', '$\mathtt{H}_\mathbf{1}$', '$\mathtt{H}_\oplus$']

box_avg_df = pd.DataFrame({
    'box' : box_df.columns,
    'average' : box_df.loc['Total'] / all_box_usage_total,
})
def addlabels(x,y):
    for i in range(len(x)):
        height = y[i] - 10 if y[i] * 100 + 1 > 10 else 0
        plt.text(i, y[i] * 100 + 1, round(y[i] * 100, ndigits=2), ha = 'center')

plt.bar(box_avg_df['box'], box_avg_df['average'] * 100, bottom=0, color='#6c8ebf')
plt.xlabel('Box used')
plt.ylabel('\% of usage in the benchmarks')
# plt.ylim(top = )
plt.ylim(0, 60)
addlabels(box_avg_df['box'], box_avg_df['average'])
# plt.show()
plt.savefig(blif_bc_output, format='pdf', dpi=300, facecolor='white', bbox_inches='tight', pad_inches=0.05)
plt.clf()


accent_color = '#6c8ebf'
title_color = '#10263b'

<Figure size 432x288 with 0 Axes>

In [77]:
def doScatterPlot(df: pd.DataFrame, ref: str, tgt: str):
    plt.scatter(df[ref], df[tgt], c=accent_color, s=5)
    plt.xlabel(f'{ref.upper()} node count')
    plt.ylabel(f'{tgt.upper()} node count')
    plt.xlim((0, 25000))
    plt.ylim((0, 25000))
    plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, ls='--', c='black')

    plt.savefig(f"./{tgt}-vs-{ref}.pdf", format='pdf', dpi=300, facecolor='white',
                edgecolor='r', bbox_inches='tight', pad_inches=0.1)
    plt.clf()

In [78]:
# doScatterPlot(df, 'zbdd', 'abdd')
# doScatterPlot(df, 'tbdd', 'abdd')
# doScatterPlot(df, 'esrbdd', 'abdd')
# doScatterPlot(df, 'czdd', 'abdd')
# doScatterPlot(df, 'cbdd', 'abdd')
# doScatterPlot(df, 'bdd', 'abdd')

color_1 = "#3dd9c1"
color_2 = "#3d5cd9"
color_3 = "#3dd954"
color_4 = "#e88700"
color_5 = "#d9453d"
color_6 = "#d93dd1"

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(df['zbdd'],   df['abdd'], c=color_2, marker='3', s=50, linewidths=1, label='ZBDD')
ax1.scatter(df['cbdd'],   df['abdd'], c=color_3, marker='+', s=50, linewidths=1, label='CBDD')
ax1.scatter(df['czdd'],   df['abdd'], c=color_4, marker='1', s=50, linewidths=1, label='CZDD')
ax1.scatter(df['tbdd'],   df['abdd'], c=color_5, marker='2', s=50, linewidths=1, label='TBDD')
ax1.scatter(df['esrbdd'], df['abdd'], c=color_6, marker='x', s=50, linewidths=1, label='ESRBDD')
ax1.scatter(df['bdd'],    df['abdd'], c=color_1, marker='4', s=50, linewidths=1, label='BDD')
plt.xlabel(f'Other BDD model node count')
plt.ylabel(f'ABDD node count')
ax1.legend()
# plt.legend(loc='upper left')
plt.xlim((0, 25000))
plt.ylim((0, 25000))
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, ls='--', c='black')

# plt.show()
plt.savefig(blif_nc_output, format='pdf', dpi=300, facecolor='white',
            edgecolor='r', bbox_inches='tight', pad_inches=0.1)
plt.clf()


<Figure size 432x288 with 0 Axes>

In [79]:
# /
# src/              # python source files
# cpp/              # C++ source files
# tests/            # unit-test files
# benchmarks/       # benchmark files
#   nta/            # from VATA repository
#   blif/           # LGSynth'91
#   dimacs/         # uf-20, uf-100 ??
# results/          # graphs, csv, etc
# data             # simulations, debugging, images, ...
# jupyter-notebooks
# LICENSE
# README.md
# requirements.txt